<a href="https://colab.research.google.com/github/CarlosIvars/APR/blob/main/2023-24/prac2/s1/s1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# S1. Red convolucional para MNIST



Como en la anterior práctica, primero importamos el conjunto de MNIST y lo normalizamos, pero sin convertir las imágenes en vectores unidimensionales, ya que vamos a trabajar con redes convolucionales que explotan la estructura 2D de las imágenes.

In [1]:
## Importar y normalizar datos

from tensorflow import keras
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('training set', x_train.shape)
print('test set', x_test.shape)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize [0..255]-->[0..1]
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
num_classes=10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

print('training set', x_train.shape)
print('val set', x_val.shape)

11490434/11490434 [==============================] - 2s 0us/step
training set (60000, 28, 28)
test set (10000, 28, 28)
training set (48000, 28, 28)
val set (12000, 28, 28)


## Modelo base
 Partiremos de una topología base que toma la red MLP de la última sesión de la primera práctica (dos capas densas de 1024 neuronas), y le incorpora un par de capas convolucionales cada una seguida por average pooling inspirada en la arquitectura LeNet (1998) propuesta por Yann LeCun para MNIST.


In [ ]:
from keras import Sequential
from keras.layers import Input, Conv2D, AveragePooling2D, Flatten, Dense
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

model = Sequential()
# ahora trabajamos con imagenes no con vectores ya que las redes convolucionales trabajan mejor asi
model.add(Input((28,28,1)))
#numero de flitros = cantidad de pesos, el kernel es el batch pero en imagen, como trozos de 5x5, en este caso se aprende 5*5*6 pesos para cada kernel.
model.add(Conv2D(filters=6, kernel_size=(5,5), activation='relu', input_shape=(28,28,1)))
#pool el promedio para 4 pixeles 2x2, se mueve dos pixeles a la derecha(strides)
model.add(AveragePooling2D(pool_size=(2,2), strides=2))
model.add(Conv2D(filters=16, kernel_size=(5,5), activation='relu'))
model.add(AveragePooling2D(pool_size=(2,2), strides=2))
#pasamos a 1 dimension para aplicar la red neuronal
model.add(Flatten())
model.add(Dense(units=1024, activation='relu'))
model.add(Dense(units=1024, activation='relu'))
model.add(Dense(units=10, activation = 'softmax'))

opt=Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

epochs=25
batch_size=128
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[reduce_lr,checkpoint])

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print(f'Test loss: {score[0]*100:.2f}')
print(f'Test accuracy: {score[1]*100:.2f}')

Epoch 1/25
369/375 [============================>.] - ETA: 0s - loss: 0.2812 - accuracy: 0.9181
Epoch 1: val_accuracy improved from -inf to 0.97050, saving model to best_model.h5
375/375 [==============================] - 3s 5ms/step - loss: 0.2782 - accuracy: 0.9189 - val_loss: 0.0968 - val_accuracy: 0.9705 - lr: 0.0010
Epoch 2/25
375/375 [==============================] - ETA: 0s - loss: 0.0738 - accuracy: 0.9773
Epoch 2: val_accuracy improved from 0.97050 to 0.98158, saving model to best_model.h5
375/375 [==============================] - 2s 5ms/step - loss: 0.0738 - accuracy: 0.9773 - val_loss: 0.0622 - val_accuracy: 0.9816 - lr: 0.0010
Epoch 3/25
374/375 [============================>.] - ETA: 0s - loss: 0.0526 - accuracy: 0.9838
Epoch 3: val_accuracy improved from 0.98158 to 0.98533, saving model to best_model.h5
375/375 [==============================] - 2s 6ms/step - loss: 0.0525 - accuracy: 0.9838 - val_loss: 0.0488 - val_accuracy: 0.9853 - lr: 0.0010
Epoch 4/25
366/375 [=====

## Ejercicio:

Probar las técnicas presentadas en la práctica 1 (dropout, batchnorm y aumento de datos) para obtener un acierto en test > 99%, incluso mejor que la obtenida con redes MLP.

In [2]:
from keras import Sequential
from keras.layers import Input, Conv2D, AveragePooling2D, Flatten, Dense,BatchNormalization

from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint


model = Sequential()
# ahora trabajamos con imagenes no con vectores ya que las redes convolucionales trabajan mejor asi
model.add(Input((28,28,1)))
#numero de flitros = cantidad de pesos, el kernel es el batch pero en imagen, como trozos de 5x5, en este caso se aprende 5*5*6 pesos para cada kernel.
model.add(Conv2D(filters=6, kernel_size=(5,5), activation='relu', input_shape=(28,28,1)))
#pool el promedio para 4 pixeles 2x2, se mueve dos pixeles a la derecha(strides)
model.add(AveragePooling2D(pool_size=(2,2), strides=2))
model.add(Conv2D(filters=16, kernel_size=(5,5), activation='relu'))
model.add(AveragePooling2D(pool_size=(2,2), strides=2))
#pasamos a 1 dimension para aplicar la red neuronal
model.add(Flatten())
model.add(Dense(units=1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(units=1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(units=10, activation = 'softmax'))

opt=Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

epochs=25
batch_size=128
history = model.fit(x_train, y_train, batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[reduce_lr,checkpoint])

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print(f'Test loss: {score[0]*100:.2f}')
print(f'Test accuracy: {score[1]*100:.2f}')

Epoch 1/25
373/375 [============================>.] - ETA: 0s - loss: 0.1551 - accuracy: 0.9553
Epoch 1: val_accuracy improved from -inf to 0.62525, saving model to best_model.h5
375/375 [==============================] - 15s 7ms/step - loss: 0.1547 - accuracy: 0.9554 - val_loss: 0.8835 - val_accuracy: 0.6252 - lr: 0.0010
Epoch 2/25
 21/375 [>.............................] - ETA: 1s - loss: 0.0585 - accuracy: 0.9784

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


374/375 [============================>.] - ETA: 0s - loss: 0.0606 - accuracy: 0.9813
Epoch 2: val_accuracy improved from 0.62525 to 0.97267, saving model to best_model.h5
375/375 [==============================] - 3s 9ms/step - loss: 0.0607 - accuracy: 0.9812 - val_loss: 0.0913 - val_accuracy: 0.9727 - lr: 0.0010
Epoch 3/25
368/375 [============================>.] - ETA: 0s - loss: 0.0417 - accuracy: 0.9865
Epoch 3: val_accuracy improved from 0.97267 to 0.98333, saving model to best_model.h5
375/375 [==============================] - 4s 11ms/step - loss: 0.0419 - accuracy: 0.9865 - val_loss: 0.0614 - val_accuracy: 0.9833 - lr: 0.0010
Epoch 4/25
375/375 [==============================] - ETA: 0s - loss: 0.0347 - accuracy: 0.9887
Epoch 4: val_accuracy improved from 0.98333 to 0.98483, saving model to best_model.h5
375/375 [==============================] - 4s 11ms/step - loss: 0.0347 - accuracy: 0.9887 - val_loss: 0.0538 - val_accuracy: 0.9848 - lr: 0.0010
Epoch 5/25
372/375 [===========

In [5]:
from keras.preprocessing.image import ImageDataGenerator


from keras import Sequential
from keras.layers import Input, Conv2D, AveragePooling2D, Flatten, Dense,BatchNormalization

from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

datagen = ImageDataGenerator(
    rotation_range=30, #de -30 a 30
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,# de -0.8 a 1.2
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest')

x_train = x_train.reshape(48000, 28, 28, 1) #con un canal, sino no funciona, modo de gris
x_val = x_val.reshape(12000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)

model = Sequential()
# ahora trabajamos con imagenes no con vectores ya que las redes convolucionales trabajan mejor asi
model.add(Input((28,28,1)))
#numero de flitros = cantidad de pesos, el kernel es el batch pero en imagen, como trozos de 5x5, en este caso se aprende 5*5*6 pesos para cada kernel.
model.add(Conv2D(filters=6, kernel_size=(5,5), activation='relu', input_shape=(28,28,1)))
#pool el promedio para 4 pixeles 2x2, se mueve dos pixeles a la derecha(strides)
model.add(AveragePooling2D(pool_size=(2,2), strides=2))
model.add(Conv2D(filters=16, kernel_size=(5,5), activation='relu'))
model.add(AveragePooling2D(pool_size=(2,2), strides=2))
#pasamos a 1 dimension para aplicar la red neuronal
model.add(Flatten())
model.add(Dense(units=1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(units=1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(units=10, activation = 'softmax'))

opt=Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

epochs=25
batch_size=128
history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[reduce_lr,checkpoint])

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print(f'Test loss: {score[0]*100:.2f}')
print(f'Test accuracy: {score[1]*100:.2f}')

Epoch 1/25
374/375 [============================>.] - ETA: 0s - loss: 0.3829 - accuracy: 0.8862
Epoch 1: val_accuracy improved from -inf to 0.60108, saving model to best_model.h5
375/375 [==============================] - 21s 47ms/step - loss: 0.3824 - accuracy: 0.8863 - val_loss: 1.2162 - val_accuracy: 0.6011 - lr: 0.0010
Epoch 2/25
374/375 [============================>.] - ETA: 0s - loss: 0.1670 - accuracy: 0.9498
Epoch 2: val_accuracy improved from 0.60108 to 0.97025, saving model to best_model.h5
375/375 [==============================] - 17s 46ms/step - loss: 0.1669 - accuracy: 0.9498 - val_loss: 0.0953 - val_accuracy: 0.9703 - lr: 0.0010
Epoch 3/25
374/375 [============================>.] - ETA: 0s - loss: 0.1256 - accuracy: 0.9603
Epoch 3: val_accuracy did not improve from 0.97025
375/375 [==============================] - 17s 46ms/step - loss: 0.1255 - accuracy: 0.9604 - val_loss: 0.1523 - val_accuracy: 0.9564 - lr: 0.0010
Epoch 4/25
374/375 [============================>.] - 

In [4]:
from keras import Sequential
from keras.layers import Input, Conv2D, AveragePooling2D, Flatten, Dense,BatchNormalization, Dropout

from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint


model = Sequential()
# ahora trabajamos con imagenes no con vectores ya que las redes convolucionales trabajan mejor asi
model.add(Input((28,28,1)))
#numero de flitros = cantidad de pesos, el kernel es el batch pero en imagen, como trozos de 5x5, en este caso se aprende 5*5*6 pesos para cada kernel.
model.add(Conv2D(filters=6, kernel_size=(5,5), activation='relu', input_shape=(28,28,1)))
#pool el promedio para 4 pixeles 2x2, se mueve dos pixeles a la derecha(strides)
model.add(AveragePooling2D(pool_size=(2,2), strides=2))
model.add(Conv2D(filters=16, kernel_size=(5,5), activation='relu'))
model.add(AveragePooling2D(pool_size=(2,2), strides=2))
#pasamos a 1 dimension para aplicar la red neuronal
model.add(Flatten())
model.add(Dense(units=1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=10, activation = 'softmax'))

opt=Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

epochs=25
batch_size=128
history = model.fit(x_train, y_train, batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[reduce_lr,checkpoint])

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print(f'Test loss: {score[0]*100:.2f}')
print(f'Test accuracy: {score[1]*100:.2f}')

Epoch 1/25
375/375 [==============================] - ETA: 0s - loss: 0.3009 - accuracy: 0.9060
Epoch 1: val_accuracy improved from -inf to 0.97392, saving model to best_model.h5
375/375 [==============================] - 6s 8ms/step - loss: 0.3009 - accuracy: 0.9060 - val_loss: 0.0875 - val_accuracy: 0.9739 - lr: 0.0010
Epoch 2/25
 24/375 [>.............................] - ETA: 1s - loss: 0.1148 - accuracy: 0.9658

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


371/375 [============================>.] - ETA: 0s - loss: 0.0960 - accuracy: 0.9715
Epoch 2: val_accuracy improved from 0.97392 to 0.97908, saving model to best_model.h5
375/375 [==============================] - 2s 5ms/step - loss: 0.0956 - accuracy: 0.9716 - val_loss: 0.0669 - val_accuracy: 0.9791 - lr: 0.0010
Epoch 3/25
367/375 [============================>.] - ETA: 0s - loss: 0.0689 - accuracy: 0.9791
Epoch 3: val_accuracy improved from 0.97908 to 0.98467, saving model to best_model.h5
375/375 [==============================] - 2s 5ms/step - loss: 0.0691 - accuracy: 0.9791 - val_loss: 0.0530 - val_accuracy: 0.9847 - lr: 0.0010
Epoch 4/25
370/375 [============================>.] - ETA: 0s - loss: 0.0561 - accuracy: 0.9827
Epoch 4: val_accuracy improved from 0.98467 to 0.98808, saving model to best_model.h5
375/375 [==============================] - 2s 5ms/step - loss: 0.0559 - accuracy: 0.9827 - val_loss: 0.0424 - val_accuracy: 0.9881 - lr: 0.0010
Epoch 5/25
367/375 [=============

datagen + dropout

In [6]:
from keras.preprocessing.image import ImageDataGenerator


from keras import Sequential
from keras.layers import Input, Conv2D, AveragePooling2D, Flatten, Dense,BatchNormalization

from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

datagen = ImageDataGenerator(
    rotation_range=30, #de -30 a 30
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,# de -0.8 a 1.2
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest')

x_train = x_train.reshape(48000, 28, 28, 1) #con un canal, sino no funciona, modo de gris
x_val = x_val.reshape(12000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)

model = Sequential()
# ahora trabajamos con imagenes no con vectores ya que las redes convolucionales trabajan mejor asi
model.add(Input((28,28,1)))
#numero de flitros = cantidad de pesos, el kernel es el batch pero en imagen, como trozos de 5x5, en este caso se aprende 5*5*6 pesos para cada kernel.
model.add(Conv2D(filters=6, kernel_size=(5,5), activation='relu', input_shape=(28,28,1)))
#pool el promedio para 4 pixeles 2x2, se mueve dos pixeles a la derecha(strides)
model.add(AveragePooling2D(pool_size=(2,2), strides=2))
model.add(Conv2D(filters=16, kernel_size=(5,5), activation='relu'))
model.add(AveragePooling2D(pool_size=(2,2), strides=2))
#pasamos a 1 dimension para aplicar la red neuronal
model.add(Flatten())
model.add(Dense(units=1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=10, activation = 'softmax'))

opt=Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

epochs=25
batch_size=128
history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[reduce_lr,checkpoint])

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print(f'Test loss: {score[0]*100:.2f}')
print(f'Test accuracy: {score[1]*100:.2f}')

Epoch 1/25
375/375 [==============================] - ETA: 0s - loss: 0.7293 - accuracy: 0.7606
Epoch 1: val_accuracy improved from -inf to 0.96108, saving model to best_model.h5
375/375 [==============================] - 23s 56ms/step - loss: 0.7293 - accuracy: 0.7606 - val_loss: 0.1311 - val_accuracy: 0.9611 - lr: 0.0010
Epoch 2/25
  1/375 [..............................] - ETA: 27s - loss: 0.4840 - accuracy: 0.8047

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


375/375 [==============================] - ETA: 0s - loss: 0.2843 - accuracy: 0.9119
Epoch 2: val_accuracy improved from 0.96108 to 0.97850, saving model to best_model.h5
375/375 [==============================] - 21s 55ms/step - loss: 0.2843 - accuracy: 0.9119 - val_loss: 0.0728 - val_accuracy: 0.9785 - lr: 0.0010
Epoch 3/25
375/375 [==============================] - ETA: 0s - loss: 0.2056 - accuracy: 0.9367
Epoch 3: val_accuracy improved from 0.97850 to 0.97950, saving model to best_model.h5
375/375 [==============================] - 17s 46ms/step - loss: 0.2056 - accuracy: 0.9367 - val_loss: 0.0702 - val_accuracy: 0.9795 - lr: 0.0010
Epoch 4/25
375/375 [==============================] - ETA: 0s - loss: 0.1737 - accuracy: 0.9470
Epoch 4: val_accuracy improved from 0.97950 to 0.98625, saving model to best_model.h5
375/375 [==============================] - 17s 46ms/step - loss: 0.1737 - accuracy: 0.9470 - val_loss: 0.0477 - val_accuracy: 0.9862 - lr: 0.0010
Epoch 5/25
374/375 [=======

solo datagen !!!!!!!!!!! el numero de decimales va en correspondencia con el numero de muestras del test para 10000 solo vale 2 decimales ya que 99.99 seria 9999 muestras un 99.992 no tiene sentido.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator


from keras import Sequential
from keras.layers import Input, Conv2D, AveragePooling2D, Flatten, Dense,BatchNormalization

from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

datagen = ImageDataGenerator(
    rotation_range=30, #de -30 a 30
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,# de -0.8 a 1.2
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest')

x_train = x_train.reshape(48000, 28, 28, 1) #con un canal, sino no funciona, modo de gris
x_val = x_val.reshape(12000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)

model = Sequential()
# ahora trabajamos con imagenes no con vectores ya que las redes convolucionales trabajan mejor asi
model.add(Input((28,28,1)))
#numero de flitros = cantidad de pesos, el kernel es el batch pero en imagen, como trozos de 5x5, en este caso se aprende 5*5*6 pesos para cada kernel.
model.add(Conv2D(filters=6, kernel_size=(5,5), activation='relu', input_shape=(28,28,1)))
#pool el promedio para 4 pixeles 2x2, se mueve dos pixeles a la derecha(strides)
model.add(AveragePooling2D(pool_size=(2,2), strides=2))
model.add(Conv2D(filters=16, kernel_size=(5,5), activation='relu'))
model.add(AveragePooling2D(pool_size=(2,2), strides=2))
#pasamos a 1 dimension para aplicar la red neuronal
model.add(Flatten())
model.add(Dense(units=1024, activation='relu'))
model.add(Dense(units=1024, activation='relu'))
model.add(Dense(units=10, activation = 'softmax'))

opt=Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

epochs=25
batch_size=128
history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[reduce_lr,checkpoint])

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print(f'Test loss: {score[0]*100:.2f}')
print(f'Test accuracy: {score[1]*100:.2f}')

Epoch 1/25
374/375 [============================>.] - ETA: 0s - loss: 0.5671 - accuracy: 0.8196
Epoch 1: val_accuracy improved from -inf to 0.96750, saving model to best_model.h5
375/375 [==============================] - 19s 46ms/step - loss: 0.5663 - accuracy: 0.8199 - val_loss: 0.1115 - val_accuracy: 0.9675 - lr: 0.0010
Epoch 2/25
375/375 [==============================] - ETA: 0s - loss: 0.2366 - accuracy: 0.9269
Epoch 2: val_accuracy improved from 0.96750 to 0.97542, saving model to best_model.h5
375/375 [==============================] - 17s 45ms/step - loss: 0.2366 - accuracy: 0.9269 - val_loss: 0.0824 - val_accuracy: 0.9754 - lr: 0.0010
Epoch 3/25
374/375 [============================>.] - ETA: 0s - loss: 0.1734 - accuracy: 0.9467
Epoch 3: val_accuracy improved from 0.97542 to 0.97592, saving model to best_model.h5
375/375 [==============================] - 17s 45ms/step - loss: 0.1732 - accuracy: 0.9468 - val_loss: 0.0791 - val_accuracy: 0.9759 - lr: 0.0010
Epoch 4/25
374/375 